**Configuration**
- Model: `VGG19`
- Dataset: `CIFAR10`

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from torch import nn
from torchvision import models
from torch.utils.data import DataLoader
from torch.optim import Adam
import datetime 
from tqdm import tqdm

In [2]:
# Hyperparameters
LEARN_RATE = 0.001
NUM_EPOCHS = 30

In [3]:
# TensorBoard setup
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(f"runs/part_a2_{current_time}")

### Step 1. Load and Transform Dataset

In [4]:
transform = transforms.Compose([
    transforms.Resize(224),  # Resize images to 224x224 for VGG19
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

Files already downloaded and verified


In [6]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

Files already downloaded and verified


### Step 2. Setup the Model

In [7]:
vgg19 = models.vgg19(pretrained=True)
vgg19

/Users/odai/cs5567-final-project/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/odai/cs5567-final-project/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [8]:
# Freeze the features part
for param in vgg19.features.parameters():
    param.requires_grad = False

In [9]:
# Modify the classifier part
vgg19.classifier[6] = nn.Linear(4096, 10)  # CIFAR10 has 10 classes

In [10]:
# Make sure we're using GPU
device = torch.device(
    "mps"  # for macOS
    if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available() else "cpu"
)
vgg19 = vgg19.to(device)


device

device(type='mps')

### Step 3. Train

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(vgg19.classifier.parameters(), lr=LEARN_RATE)  # Only train the classifier parameters

In [12]:
def train_model(model, criterion, optimizer, num_epochs):
    model.train()
    for epoch in tqdm(range(num_epochs)):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100}')
                writer.add_scalar('training loss', running_loss / 100, epoch * len(trainloader) + i)
                running_loss = 0.0
    print('Finished Training')

train_model(vgg19, criterion, optimizer, NUM_EPOCHS)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1, Batch 100, Loss: 1.1575207507610321
Epoch 1, Batch 200, Loss: 0.9012425389885902
Epoch 1, Batch 300, Loss: 0.8967584544420242
Epoch 1, Batch 400, Loss: 0.9230339881777764
Epoch 1, Batch 500, Loss: 0.8562506744265557
Epoch 1, Batch 600, Loss: 0.8723183608055115
Epoch 1, Batch 700, Loss: 0.864463771879673
Epoch 1, Batch 800, Loss: 0.8170809775590897
Epoch 1, Batch 900, Loss: 0.8235704334080219
Epoch 1, Batch 1000, Loss: 0.814619879424572
Epoch 1, Batch 1100, Loss: 0.849081598520279
Epoch 1, Batch 1200, Loss: 0.8522417584061622
Epoch 1, Batch 1300, Loss: 0.8998754665255546
Epoch 1, Batch 1400, Loss: 0.7803656539320946


  0%|          | 0/30 [33:39<?, ?it/s]


KeyboardInterrupt: 